In [2]:
%%sh 
pip install pyarrow

     --------------------------------------- 24.3/24.3 MB 15.6 MB/s eta 0:00:00


In [3]:
%%sh 
pip install fastparquet

     ------------------------------------- 715.2/715.2 kB 22.0 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 21.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pyarrow
import fastparquet
import pickle

with open('trained_model.pkl', 'rb') as f: 
    model = pickle.load(f)
    

    
messy_data = pd.read_csv('./troop_movements10m.csv')

messy_data.loc[messy_data['unit_type']=='invalid_unit', 'unit_type'] ='unknown'

messy_data.location_x.interpolate (method='ffill', inplace=True)
messy_data.location_y.interpolate (method='ffill', inplace=True)

messy_data.to_parquet('clean_data.parquet',engine="fastparquet")

In [3]:
clean_data = pd.read_parquet('./clean_data.parquet')

X = clean_data[['unit_type', 'homeworld']]
X_encoded = pd.get_dummies(X)

predictions = model.predict(X_encoded)


[ True False False ... False False False]


In [4]:
clean_data['predictions'] = predictions
clean_data

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld,predictions
0,2023-06-13 17:33:18,1,at-st,2.0,8.0,1,1,Glee Anselm,True
1,2023-06-13 17:33:17,2,tie_silencer,4.0,4.0,0,1,Trandosha,False
2,2023-06-13 17:33:16,3,at-at,0.0,3.0,6,1,Corellia,False
3,2023-06-13 17:33:15,4,tie_silencer,6.0,1.0,6,9,Shili,True
4,2023-06-13 17:33:14,5,tie_fighter,0.0,4.0,9,6,Muunilinst,False
...,...,...,...,...,...,...,...,...,...
9999995,2023-02-18 10:26:20,9999996,resistance_soldier,6.0,6.0,1,3,Troiken,False
9999996,2023-02-18 10:26:19,9999997,tie_silencer,8.0,6.0,2,0,Kashyyyk,True
9999997,2023-02-18 10:26:18,9999998,tie_fighter,7.0,7.0,6,4,Kashyyyk,False
9999998,2023-02-18 10:26:17,9999999,tie_fighter,6.0,6.0,8,8,Kalee,False
